In [1]:
import Toy_System_Fully_Continuous as ts
import numpy as np
from multi_step import *
from utils import objective, stage

Using backend: tensorflow


In [2]:
# Generate the toy problem class object

toy = ts.Toy()

In [3]:
# Generate conditions to be used for generating data

# Define the condition ranges [lower,upper,step (if applicable)]
#[T1, P1, t_out1, T2, P2, t_out2, T3, P3, t_out3]
condition_ranges = [[2,10],[2,10],[2,10],[2,10],[2,10],[2,10],[2,10],[2,10],[2,10]]

# Generate all condition permutations
# conditions = toy.allConditions(condition_ranges)

# Generate values to explore the search space using low discrepancy
# quasi-random numbers and discrete methods
num_points = 1000
conditions =  toy.sobolSequenceConditions(condition_ranges, num_points)

In [4]:
# Generate the yields of the various materials
yieldList = []
costList = []
for conditions_run in conditions:
    yields, costs = toy.experimentalRun(1,conditions_run)
    yieldList.append(yields)
    costList.append(costs)
yieldList = np.array(yieldList)

In [5]:
# Fit the first stage data
x1 = conditions[:,:3]
y1 = yieldList[:,2:5]
toy.fit_stage_1([x1,y1], epochs=5)

epoch:  0 , loss =  6.2280874


In [6]:
# Fit the second stage data

# x2 = np.concatenate((np.expand_dims(yieldList[:,1],axis=1),np.expand_dims(yieldList[:,3],axis=1),conditions[:,3:6]),axis=1)
x2 = np.concatenate((np.expand_dims(yieldList[:,3],axis=1),conditions[:,3:6]),axis=1)
y2 = yieldList[:,5:9]
toy.fit_stage_2([x2,y2], epochs=5)

epoch:  0 , loss =  15.308417


In [7]:
# Fit the third stage data
x3 = np.concatenate((np.expand_dims(yieldList[:,4],axis=1),yieldList[:,7:9],conditions[:,6:]),axis=1)
y3 = yieldList[:,9:]
toy.fit_stage_3([x3,y3], epochs=5)

epoch:  0 , loss =  7.583346


In [8]:
#Plot stage 1
# num_plot_points = 20
# plot_var = 0
# plot_x = np.hstack((np.expand_dims(np.linspace(2,10,num_plot_points),1),np.full((num_plot_points,1),7),np.full((num_plot_points,1),5)))
# plot_y = np.array([toy.stage_1(0.4, *row) for row in plot_x])
# toy.stage_1_model.visualise(plot_x,plot_y,plot_x,plot_var,'Stage_1')

In [9]:
# Save the models
# toy.stage_1_model.save('../examples/Toy_problem/Saved_Models/stage_1_model')
# toy.stage_2_model.save('../examples/Toy_problem/Saved_Models/stage_2_model')
# toy.stage_3_model.save('../examples/Toy_problem/Saved_Models/stage_3_model')

In [10]:
# Generate the multi-step class
multi_step = multi_step('adam')

In [11]:
# Load the models from a save
# import tensorflow as tf
# toy = ts.Toy()
# 
# toy.stage_1_model = tf.keras.models.load_model('../examples/Toy_problem/Saved_Models/stage_1_model.h5')
# toy.stage_2_model = tf.keras.models.load_model('../examples/Toy_problem/Saved_Models/stage_2_model.h5')
# toy.stage_3_model = tf.keras.models.load_model('../examples/Toy_problem/Saved_Models/stage_3_model.h5')

In [12]:
# Load the models into the framework

#Define the stages
stage1 = stage(0, toy.stage_1_model, condition_ranges[0:3], [[1, [1]], [2, [2]]])
stage2 = stage(1, toy.stage_2_model, condition_ranges[3:6], [[2,[2,3]]])
stage3 = stage(2, toy.stage_3_model, condition_ranges[6:])

stages = [stage1, stage2, stage3]
# list of models
# stages = [toy.stage_1_model, toy.stage_2_model, toy.stage_3_model]
# IDs for the models
# stageIDs = [1,2,3]
# Connectivity of the models
# stageConnectivity = np.array([[stageIDs[0],stageIDs[1]],[stageIDs[0],stageIDs[2]],[stageIDs[1],stageIDs[2]]])
# Stage input ranges
# inputRanges = condition_ranges

# multi_step.loadModels(stages, stageIDs, stageConnectivity)
multi_step.loadModels(stages)

C:\Users\geeso\Anaconda3\envs\multi-step\lib\site-packages\dgl\base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


In [13]:
# Define the objective functions
def objectiveFunction1(var1, var2, var3):
    return (var1+var2)**2+(var2+var3)**2
    
def objectiveFunction2(var1, var2, var3, var4):
    return (var1-var4)**0.5-var3**2+var2**1.5

# Define the objective objects including the objective functions and the IDs of relevant variables/outputs
# Define objective variables with form: [stage, 'input'/'output', stage variable]
# objective1 = objective(objectiveFunction1(), [[stageIDs[0],1],[stageIDs[1],1],[stageIDs[2],1]])
# objective2 = objective(objectiveFunction2(),[[stageIDs[0],2],[stageIDs[1],2],[stageIDs[2],2]])
objective1 = objective(objectiveFunction1, [[stage1,'inputs',1],[stage2,'inputs',1],[stage3,'inputs',1]])
objective2 = objective(objectiveFunction2,[[stage1,'inputs',2],[stage2,'inputs',2],[stage3,'inputs',2],[stage2,'outputs',1]])

objectives = [objective1, objective2]

In [14]:
# Define the objectives for the framework
multi_step.defineObjectives(objectives)

In [15]:
# Optimise the process
# numObjectives, preferenceCodeDim, hyperparameterGeneratorDim, epochs=5, h_dim=5, n_layers=4, network_type='GATConv', **kwargs
multi_step.optimise(2, 4, 5, 5, 5, 4, 'GATConv', num_heads=3)

dict_values([[TensorShape([1, 3, 6]), TensorShape([1, 3, 6]), TensorShape([6, 18])], [TensorShape([1, 3, 6]), TensorShape([1, 3, 6]), TensorShape([6, 18])], [TensorShape([1, 3, 6]), TensorShape([1, 3, 6]), TensorShape([6, 18])], [TensorShape([1, 3, 6]), TensorShape([1, 3, 6]), TensorShape([6, 18])]])

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Hyperparams
dict_values([[<tf.Tensor: shape=(1, 3, 6), dtype=float32, numpy=
array([[[-0.01333961, -0.14102748,  0.0573674 ,  0.04217948,
         -0.07533845, -0.00744099],
        [ 0.08643533, -0.1397129

ValueError: No gradients provided for any variable: ['dense_24/kernel:0', 'dense_24/bias:0', 'dense_25/kernel:0', 'dense_25/bias:0', 'dense_26/kernel:0', 'dense_26/bias:0', 'dense_27/kernel:0', 'dense_27/bias:0', 'dense_28/kernel:0', 'dense_28/bias:0', 'dense_29/kernel:0', 'dense_29/bias:0', 'dense_34/kernel:0', 'dense_34/bias:0', 'dense_35/kernel:0', 'dense_35/bias:0', 'dense_36/kernel:0', 'dense_36/bias:0'].

In [ ]:
# import networkx as nx
# G = multi_step.stageGraph.to_networkx()
# nx.draw(G)